## Model: distilgpt2

In [1]:
%pip uninstall -y pinecone-client
%pip install pinecone-client==3.0.0 --quiet

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
import pinecone
from dotenv import load_dotenv
import os
import torch
import logging

In [2]:
load_dotenv()
logging.basicConfig(level=logging.INFO)
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = "youtube-transcripts"

In [3]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [8]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX_NAME)

In [9]:
def retrieve_chunks(query, top_k=5):
    query_embedding = embedder.encode(query, show_progress_bar=False).tolist()
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)
    return [match["metadata"]["text"] for match in results["matches"]]

In [10]:
def generate_response(query, chunks):
    context = "\n".join(chunks)
    prompt = f"Answer the question based on the context:\nContext: {context}\nQuestion: {query}"
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=150)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [11]:
test_queries = [
    "What is ITSM in ServiceNow?",
    "Explain CMDB relationships.",
    "How does Incident Management work?"
]

In [12]:
for query in test_queries:
    try:
        chunks = retrieve_chunks(query)
        response = generate_response(query, chunks)
        print(f"Q: {query}\nA: {response}\n")
    except Exception as e:
        print(f"Error for query '{query}': {e}")

Token indices sequence length is longer than the specified maximum sequence length for this model (819 > 512). Running this sequence through the model will result in indexing errors


Q: What is ITSM in ServiceNow?
A: aiops and event management

Q: Explain CMDB relationships.
A: a.

Q: How does Incident Management work?
A: proactive versus reactive

